##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fundamentos de la personalización: tensores y operaciones

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/basics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a> </td>
</table>

Este es un tutorial de introducción a TensorFlow que muestra cómo:

- Importar el paquete deseado.
- Crear y usar tensores.
- Usar la aceleración por GPU.
- Construir una canalización de datos con  `tf.data.Dataset`.

## Importar TensorFlow

El primer paso es importar el módulo `tensorflow`. A partir de TensorFlow 2, eager execution está habilitado por default. Eager execution permite un front-end más interactivo para TensorFlow, que podrá ver con más detalle más adelante.

In [ ]:
import tensorflow as tf

## Tensores

Los tensores son matrices multidimensionales. Como los objetos `ndarray` de NumPy, los objetos `tf.Tensor` tienen un tipo de datos y una forma. Además, los `tf.Tensor` pueden residir en la memoria del acelerador (como una GPU). TensorFlow provee una rica librería de operaciones (por ejemplo, `tf.math.add`, `tf.linalg.matmul`, y `tf.linalg.inv`) que consumen y producen `tf.Tensor`es. Estas operaciones convertirán automáticamente los tipos incorporados de Python. Por ejemplo:


In [ ]:
print(tf.math.add(1, 2))
print(tf.math.add([1, 2], [3, 4]))
print(tf.math.square(5))
print(tf.math.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.math.square(2) + tf.math.square(3))

Cada `tf.Tensor` tiene una forma y un tipo de datos:

In [ ]:
x = tf.linalg.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

Las diferencias más obvias entre los `tf.Tensor` y los arreglos NumPy son:

1. Los tensores pueden encontrarse en la memoria aceleradora (como GPU, TPU).
2. Los tensores son inmutables.

### Compatibilidad con NumPy

Es fácil convertir entre un `tf.Tensor` TensorFlow y un `ndarray` NumPy:

- Las operaciones TensorFlow convierten automáticamente los ndarrays NumPy en Tensores.
- Las operaciones NumPy convierten automáticamente los Tensores en ndarrays NumPy.

Los tensores se convierten explícitamente en ndarrays NumPy usando su método `.numpy()`. Dado que el arreglo y el `tf.Tensor` comparten la representación de memoria subyacente cuando es posible, estas conversiones suelen ser económicas. Pero compartir la representación subyacente no siempre es posible, ya que el `tf.Tensor` puede estar en la memoria de la GPU, mientras que las matrices de NumPy siempre se encuentran en la memoria del host, y la conversión implica una copia de la GPU a la memoria del host.

In [ ]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.math.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to NumPy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

## Aceleración por GPU

Muchas operaciones de TensorFlow usan GPU para acelerar la computación. En ausencia de anotaciones, TensorFlow decide automáticamente si usar la GPU o la CPU para una operación, copiando el tensor entre la memoria de la CPU y la de la GPU si es necesario. Los tensores generados por una operación se almacenarán normalmente en la memoria del dispositivo que realizó el proceso. Por ejemplo:

In [ ]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

### Nombres de dispositivos

La propiedad `Tensor.device` indica un nombre de cadena completamente calificado del dispositivo en el que se alojan los contenidos del Tensor. Este nombre condensa muchos detalles, como un identificador para la dirección de red del host en el que se está ejecutando este programa, y el dispositivo dentro de ese host. Un programa TensorFlow lo necesita para su ejecución distribuida. La cadena termina con `GPU:<N>` si el tensor se encuentra en la GPU número `N` del host.

### Colocación explícita en el dispositivo

En TensorFlow, la *colocación* se refiere a cómo se asignan (colocan) las operaciones individuales a un dispositivo para su ejecución. Como se mencionó anteriormente, si no se ofrece ninguna orientación explícita, TensorFlow decidirá automáticamente en qué dispositivo debe ejecutarse una operación, y copiará los tensores a ese dispositivo si es necesario.

No obstante, las operaciones TensorFlow pueden colocarse explícitamente en dispositivos específicos usando el gerente de contexto del `tf.device`. Por ejemplo:

In [ ]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.linalg.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

## Conjuntos de datos

Esta sección usa la API `tf.data.Dataset` para construir una canalización que suministre datos a tu modelo. `tf.data.Dataset` se usa para construir canalizaciones de entrada complejas y eficaces a partir de piezas sencillas y reutilizables que suministrarán datos a los bucles de entrenamiento o evaluación de tu modelo. (Consulte la sección [tf.data: Construir canalizaciones de entrada de TensorFlow](../../guide/data.ipynb) para saber más).

### Crear un `Dataset` fuente

Cree un conjunto de datos *fuente* usando una de las funciones de fábrica como `tf.data.Dataset.from_tensors`, `tf.data.Dataset.from_tensor_slices`, o usando objetos que lean de archivos como `tf.data.TextLineDataset` o `tf.data.TFRecordDataset`. Consulte la sección *Lectura de datos de entrada* de la guía [tf.data: Construir canalizaciones de entrada de TensorFlow](../../guide/data.ipynb) para saber más.

In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

### Aplicar transformaciones

Use las funciones de transformación como `tf.data.Dataset.map`, `tf.data.Dataset.batch`, y `tf.data.Dataset.shuffle` para aplicar transformaciones a los registros del conjunto de datos.

In [ ]:
ds_tensors = ds_tensors.map(tf.math.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

### Iterar

Los objetos `tf.data.Dataset` admiten la iteración para recorrer los registros:

In [ ]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)